# Create `MBTiles`
In this notebook we will transforms vector type data into MBTiles using tippecanoe.

# Setup
## Library import
We import all the required Python libraries

In [3]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess
from dotenv import load_dotenv
import glob
from google.cloud import storage
from google.oauth2 import service_account
import ipyleaflet as ipyl

## Utils

**read_indicators**

In [4]:
def read_indicators(dataset, indicator, gdf_vectors, geo_type='Municipios', filter_type = 'year'):
     gdf_ids = {'Provincias': ['CO_CCAA', 'CO_PROVINC'],
     'Comunidades autonomas': ['CO_CCAA'],
     'Municipios': ['CODIGOINE'],
     'Rejilla': ['grid_id']}

     gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
     'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
     'Municipios': ['NAMEUNIT', 'CODIGOINE'],
     'Rejilla': ['grid_id']}

     geo_name = geo_type.lower().replace(' ', '_')

     df = pd.read_csv(f'../../datasets/processed/{dataset}/{dataset}_{geo_name}.csv')
     df = df.astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type]))))
     gdf = df[(df['indicator'] == indicator)]
     gdf = gpd.GeoDataFrame(pd.merge(gdf, gdf_vectors[geo_type][gdf_columns[geo_type]+['geometry']].astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type])))),\
          how='left', on=gdf_ids[geo_type]))


     if filter_type == 'cereal':
          gdf = gdf[gdf_columns[geo_type] + ['dataset', 'indicator', 'scenario', 'value', filter_type, 'unit', 'geometry']]
     else:
          gdf = gdf[gdf_columns[geo_type] + ['dataset', 'indicator', 'scenario', 'value', 'year', 'unit', 'geometry']]

     return gdf

**reorganize_data_new**

In [5]:
def reorganize_data_new(gdf, geo_type='Municipios', filter_type = 'year'):
    gdf_id = {'Provincias': 'CO_PROVINC',
    'Comunidades autonomas': 'CO_CCAA',
    'Municipios': 'CODIGOINE',
    'Rejilla': 'grid_id'}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE'],
    'Rejilla': ['grid_id']}

    gdf_new = gpd.GeoDataFrame(columns=gdf_columns[geo_type]+['dataset', 'indicator', 'unit', 'geometry'])
    for code_id in gdf[gdf_id[geo_type]].unique():
        df = gdf[gdf[gdf_id[geo_type]] == code_id][['scenario', filter_type, 'value']]

        gdf_tmp = gdf[gdf[gdf_id[geo_type]] == code_id][gdf_columns[geo_type]].iloc[:1]
        gdf_tmp['dataset'] = gdf['dataset'].iloc[0]
        gdf_tmp['indicator'] = gdf['indicator'].iloc[0]
        gdf_tmp['unit'] = gdf['unit'].iloc[0]

        columns = []
        for  i in range(len(df)):
            filter = df[filter_type].iloc[i].replace(" ", "")
            gdf_tmp[f"value_{df['scenario'].iloc[i]}_{filter}"] = df['value'].iloc[i]

        gdf_tmp['geometry'] = gdf[gdf[gdf_id[geo_type]] == code_id]['geometry'].iloc[:1]

        gdf_new = pd.concat([gdf_new, gdf_tmp])

    return gdf_new

**reorganize_data**

In [6]:
def reorganize_data(gdf, geo_type='Municipios', filter_type = 'year'):
    gdf_id = {'Provincias': 'CO_PROVINC',
    'Comunidades autonomas': 'CO_CCAA',
    'Municipios': 'CODIGOINE',
    'Rejilla': 'grid_id'}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE'],
    'Rejilla': ['grid_id']}

    gdf_new = gpd.GeoDataFrame(columns=gdf_columns[geo_type]+['dataset', 'indicator', 'unit', 'values', 'geometry'])
    for code_id in gdf[gdf_id[geo_type]].unique():
        LD = json.loads(gdf[gdf[gdf_id[geo_type]] == code_id][['scenario', filter_type, 'value']].to_json(orient='records'))
        if geo_type == 'Rejilla':
            v = {k: {dic[filter_type]: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['baseline', 'rcp45', 'rcp85']}
        else:
            v = {k: {dic[filter_type]: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['rcp45', 'rcp85']}

        gdf_tmp = gdf[gdf[gdf_id[geo_type]] == code_id][gdf_columns[geo_type]].iloc[:1]
        gdf_tmp['dataset'] = gdf['dataset'].iloc[0]
        gdf_tmp['indicator'] = gdf['indicator'].iloc[0]
        gdf_tmp['unit'] = gdf['unit'].iloc[0]
        gdf_tmp['values'] = [v]
        gdf_tmp['geometry'] = gdf[gdf[gdf_id[geo_type]] == code_id]['geometry'].iloc[:1]

        gdf_new = pd.concat([gdf_new, gdf_tmp])

    return gdf_new

**create_mbtiles**

In [7]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**mbtile_to_pbf**

In [8]:
def mbtile_to_pbf(input_mbtiles, output_path):
    """
    Use mb-util to convert mbtiles to to pbf in z/x/y.pbf form.
    To export MBTiles to disk, specify a directory that does not yet exist.
    """
    cmd = f"mb-util {input_mbtiles} {output_path} --image_format=pbf"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**upload_local_directory_to_gcs**

In [9]:
def upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path):
    """Uploads a directory to the bucket."""
    
    credentials_dict = {
          "type": "service_account",
          "project_id": os.getenv("PROJECT_ID"),
         "private_key_id": os.getenv("PRIVATE_KEY_ID"),
          "private_key":os.getenv("PRIVATE_KEY"),
         "client_email": os.getenv("CLIENT_EMAIL"),
          "token_uri":os.getenv("TOKEN_URI"),   
    } 
    

    credentials = service_account.Credentials.from_service_account_info(credentials_dict)
    storage_client = storage.Client(project='project_id', credentials=credentials)
    
    bucket = storage_client.bucket(bucket_name)
    rel_paths = glob.glob(local_path + '/**', recursive=True)

    for local_file in rel_paths:
        remote_path = f'{destination_blob_path}{"/".join(local_file.split(os.sep)[len(local_path.split(os.sep))-1:])}'
        if os.path.isfile(local_file):
            blob = bucket.blob(remote_path)
            print(
                "File {} uploaded to {}.".format(
                    local_file, remote_path
                )
            )
            blob.upload_from_filename(local_file)

**vector_tile_processing**

In [10]:
def vector_tile_processing(path, dataset, folder_name, gdf, geo_type='Municipios', pbf_format=True, upload_to_GCS=True):
    geo_name = geo_type.lower().replace(' ', '_')
    
    # Create output folder
    try:
        os.mkdir(f"{path}/{folder_name}")
    except OSError:
        print ("Creation of the directory %s failed" % folder_name)
    else:
        print ("Successfully created the directory %s " % folder_name)

    # Save GeoDataFrame as `GeoJSON`
    with open(f"{path}/{folder_name}/{geo_name}.json", 'w') as outfile:
        outfile.write(gdf.to_json())

    # Create MBTiles
    print('Creating MBTiles')
    source_path = f"{path}/{folder_name}/{geo_name}.json"
    mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
    #create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
    create_mbtiles(source_path, mbtil_path, folder_name, opts="-Z1 -z10 --force")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    print('Converting from MBTiles to pbf')
    if pbf_format:
        output_path = f"{path}/{folder_name}/{geo_name}"
        mbtile_to_pbf(mbtil_path, output_path)

    # Change folder permission
    os.chmod(f"{path}/{folder_name}/{geo_name}", 0o777)

    # Rename files from `.pbf` to `.vector.pbf.gz`
    for root, dirs, files in os.walk(f"{path}/{folder_name}/{geo_name}"):
        for file in files:
            if file.endswith(".pbf"):
                in_file = os.path.join(root,file)
                os.rename(in_file, in_file.replace('.pbf', '.vector.pbf.gz'))

    # gunzip folder
    cmd = f"gunzip -r {path}/{folder_name}/{geo_name}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    if upload_to_GCS:
        load_dotenv()

        bucket_name = 'ecf-agricultural-climate-impact'
        local_path = f"{path}/{folder_name}/{geo_name}"
        destination_blob_path = f"MBTiles/{folder_name}/"

        upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**all_processing**

In [11]:
def all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types = ['Provincias', 'Municipios']):
    for geo_type in geo_types:
        print(geo_type)
        # Read data
        gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

        # Reorganice data
        gdf = reorganize_data_new(gdf, geo_type) 

        # Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format
        vector_tile_processing(path, dataset, folder_name, gdf, geo_type, pbf_format=True, upload_to_GCS=True)

# Data import

## Comunidades autonomas

In [12]:
comunidades = gpd.read_file(f'../../datasets/processed/comunidades.geojson')
comunidades.sort_values(['CO_CCAA'], inplace = True)

## Provincias

In [13]:
provincias = gpd.read_file(f'../../datasets/raw/georegions/Provincias/Provincias.shp')
provincias.sort_values(['CO_CCAA', 'CO_PROVINC'], inplace = True)

## Municipios de España

In [14]:
municipios = gpd.read_file(f'../../datasets/raw/georegions/Municipios/Municipios_IGN.shp')
municipios.sort_values(['CODNUT1', 'CODNUT2', 'CODNUT3', 'CODIGOINE'], inplace = True)

# Remove Canarias, Ceuta, and Melilla
municipios = municipios[~municipios['CODNUT2'].isin(['ES70', 'ES63', 'ES64'])]
municipios = municipios.reset_index(drop=True)

## Rejilla viñedos

In [15]:
rejilla = gpd.read_file('../../datasets/processed/rejilla_viñedos.geojson')
rejilla.sort_values(['grid_id'], inplace = True)

**Put all vector data together** 

In [16]:
gdf_vectors = {'Comunidades autonomas': comunidades,
"Provincias": provincias,
"Municipios": municipios,
"Rejilla": rejilla}

# Create tiles

## Aumento de temperaturas proyectadas por municipio
**Create all tiles**

In [ ]:
dataset = 'climate_indicators'
indicator = 'mean_Tmean_Yearly_change'
path = '../../datasets/processed/MBTiles'
folder_name = 'Aumento_temperaturas'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Upload files to GCS**

In [ ]:
for geo_type in geo_types:
    print(geo_type)
    geo_name = geo_type.lower().replace(' ', '_')
    # Upload files
    
    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/{geo_name}"
    destination_blob_path = f"MBTiles/{folder_name}/"
    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**Create `MBTile` for a single `scenario` and `year`**

In [ ]:
dataset = 'climate_indicators'
indicator = 'mean_Tmean_Yearly_change'
path = '../../datasets/processed/MBTiles'
folder_name = 'Aumento_temperaturas'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)
gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2035 - 2065')]
gdf.head()

In [ ]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

## Duración de las sequías
**Create all tiles**

In [ ]:
dataset = 'climate_indicators'
indicator = 'dry-spells_maximum-length'
path = '../../datasets/processed/MBTiles'
folder_name = 'Duracion_sequias'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Upload files to GCS**

In [ ]:
for geo_type in geo_types:
    print(geo_type)
    geo_name = geo_type.lower().replace(' ', '_')
    # Upload files
    
    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/{geo_name}"
    destination_blob_path = f"MBTiles/{folder_name}/"
    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**Create `MBTile` for a single `scenario` and `year`**

In [368]:
dataset = 'climate_indicators'
indicator = 'dry-spells_maximum-length'
path = '../../datasets/processed/MBTiles'
folder_name = 'Duracion_sequias'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2041 - 2060')]
gdf.head()

,NAMEUNIT,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
4,Abegondo,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.331772,2041 - 2060,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
11,Ames,15002,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.904419,2041 - 2060,days,"POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
18,Aranga,15003,Bioclimatic indicators,dry-spells_maximum-length,rcp45,33.225460,2041 - 2060,days,"POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
25,Ares,15004,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.112511,2041 - 2060,days,"MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
32,Arteixo,15005,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.188190,2041 - 2060,days,"MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


In [460]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Duracion_sequias/municipios.mbtiles -l Duracion_sequias -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Duracion_sequias/municipios.json


8115 features, 10702553 bytes of geometry, 52417 bytes of separate metadata, 329422 bytes of string pool


Task created
Finished processing


## Mapa de cultivos
**Read data**

In [82]:
gdf_corine = gpd.read_file('../../datasets/raw/crops/CLC2018_GDB/CLC2018_ES.gdb', driver="FileGDB", layer='CLC18_ES')
# Re-Project
gdf_corine = gdf_corine.to_crs("EPSG:4326")
gdf_corine = gdf_corine.set_crs("EPSG:4326")

**Reorganice data**

In [21]:
dic = {'Cereal': ['211', '212'],
'Olivar': ['223'],
'Dehesa': ['244'],
'Viñedo': ['221']}

gdf = gpd.GeoDataFrame(columns=['ID', 'value', 'geometry'])

for crop in dic.keys():
    print(crop)
    gdf_tmp = gdf_corine[gdf_corine['CODE_18'].isin(dic[crop])].copy()
    gdf_tmp['value'] = crop
    gdf_tmp = gdf_tmp[['ID', 'value', 'geometry']]
    
    gdf  = pd.concat([gdf, gdf_tmp])

gdf['dataset'] = 'Mapa de cultivos'
gdf['indicator'] = 'Localización cultivo'

gdf = gdf[['ID', 'dataset', 'indicator', 'value', 'geometry']]

gdf.head()

Cereal
Olivar
Dehesa
Viñedo


,ID,dataset,indicator,value,geometry
16379,16382,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.64246 36.12721, -5.64226 36..."
16380,16383,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50050 36.17831, -5.50045 36..."
16381,16384,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50964 36.18659, -5.50953 36..."
16382,16385,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.69228 36.18296, -5.69232 36..."
16383,16386,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.46238 36.20897, -5.46238 36..."


In [23]:
dic = {'Cereal': 2,
'Olivar': 1,
'Dehesa': 4,
'Viñedo': 3}
gdf['crop_name'] = gdf['value']
gdf['value'] = gdf['crop_name'].apply(lambda x: dic[x])
gdf

,ID,dataset,indicator,value,geometry,crop_name
16379,16382,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.64246 36.12721, -5.64226 36...",Cereal
16380,16383,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.50050 36.17831, -5.50045 36...",Cereal
16381,16384,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.50964 36.18659, -5.50953 36...",Cereal
16382,16385,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.69228 36.18296, -5.69232 36...",Cereal
16383,16386,Mapa de cultivos,Localización cultivo,2,"MULTIPOLYGON (((-5.46238 36.20897, -5.46238 36...",Cereal
...,...,...,...,...,...,...
48671,48674,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-6.66044 42.73515, -6.66040 42...",Viñedo
48672,48675,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-1.78326 42.79147, -1.78299 42...",Viñedo
48673,48676,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-7.62373 42.80460, -7.62367 42...",Viñedo
48674,48677,Mapa de cultivos,Localización cultivo,3,"MULTIPOLYGON (((-2.65395 43.22925, -2.65397 43...",Viñedo


In [ ]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Mapa_cultivos'
name = 'Mapa_cultivos'
geo_name= 'Mapa_cultivos'

# Save GeoDataFrame as `GeoJSON`
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, folder_name, opts="-Z1 -z10 --drop-densest-as-needed --force")

**Upload files to GCS**

In [10]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Mapa_cultivos'
name = 'Mapa_cultivos'
geo_name= 'Mapa_cultivos'

In [ ]:
bucket_name = 'ecf-agricultural-climate-impact'
local_path = f"{path}/{folder_name}"
destination_blob_path = f"MBTiles/"
upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [ ]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Mapa_cultivos'
name = 'Mapa_cultivos'

vector_tile_processing(path, name, folder_name, gdf, pbf_format=True, upload_to_GCS=True)

## Proyecciones de rendimiento del olivo.
**Create all tiles**

In [ ]:
dataset = 'olive_grove_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento_olivar'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Upload files to GCS**

In [ ]:
for geo_type in geo_types:
    print(geo_type)
    geo_name = geo_type.lower().replace(' ', '_')
    # Upload files
    
    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/{geo_name}"
    destination_blob_path = f"MBTiles/{folder_name}/"
    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**Create `MBTile` for a single `scenario` and `year`**

In [369]:
dataset = 'olive_grove_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento_olivar'
geo_type = 'Municipios'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2041–2070')]
gdf.head()

,NAMEUNIT,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
0,Abegondo,15001,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
1,Ames,15002,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
2,Aranga,15003,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
3,Ares,15004,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
4,Arteixo,15005,Olivar,yield_change_per,rcp45,NaN,2041–2070,%,"MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


In [29]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z12 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Proyecciones_rendimiento/municipios.mbtiles -l Proyecciones_rendimiento -Z1 -z12 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Proyecciones_rendimiento/municipios.json


8115 features, 10691777 bytes of geometry, 49924 bytes of separate metadata, 203346 bytes of string pool


Task created
Finished processing


  99.9%  12/2022/1598  


## Cambios en zonas óptimas para la producción de vino de calidad.

In [ ]:
dataset = 'vineyard_indicators'
indicator = 'zonas_alto_potencial_climático'
path = '../../datasets/processed/MBTiles'
folder_name = 'Zonas_alto_potencial_climático_viñedo'
geo_types = ['Rejilla']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

**Upload files to GCS**

In [ ]:
for geo_type in geo_types:
    print(geo_type)
    geo_name = geo_type.lower().replace(' ', '_')
    # Upload files
    
    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/{geo_name}"
    destination_blob_path = f"MBTiles/{folder_name}/"
    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**Create `MBTile` for a single `scenario` and `year`**

In [522]:
dataset = 'vineyard_indicators'
indicator = 'zonas_alto_potencial_climático'
path = '../../datasets/processed/MBTiles'
folder_name = 'Zonas_alto_potencial_climático_viñedo'
geo_type = 'Rejilla'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

gdf = gdf[(gdf['scenario'] == 'rcp45') & (gdf['year'] == '2021-2050')]
gdf.head()

,grid_id,dataset,indicator,scenario,value,year,unit,geometry
1402,62,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,NaN,2021-2050,NaN,"POLYGON ((-3.90000 37.10000, -3.70000 37.10000..."
1403,63,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,NaN,2021-2050,NaN,"POLYGON ((-3.70000 37.10000, -3.50000 37.10000..."
1404,64,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,7.0,2021-2050,NaN,"POLYGON ((-3.50000 37.10000, -3.30000 37.10000..."
1405,65,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,7.0,2021-2050,NaN,"POLYGON ((-3.30000 37.10000, -3.10000 37.10000..."
1406,66,Cambio climático en el viñedo,zonas_alto_potencial_climático,rcp45,12.0,2021-2050,NaN,"POLYGON ((-3.10000 37.10000, -2.90000 37.10000..."


In [523]:
# Save GeoDataFrame as `GeoJSON`
geo_name = geo_type.lower().replace(' ', '_')
with open(f"{path}/{folder_name}/{geo_name}.json", 'w') as outfile:
    outfile.write(gdf.to_json())

# Create MBTiles
print('Creating MBTiles')
layer_name = folder_name
source_path = f"{path}/{folder_name}/{geo_name}.json"
mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
#create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")
create_mbtiles(source_path, mbtil_path, layer_name, opts="-Z1 -z10 --drop-densest-as-needed --force")

Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.mbtiles -l Zonas_alto_potencial_climático_viñedo -Z1 -z10 --drop-densest-as-needed --force ../../datasets/processed/MBTiles/Zonas_alto_potencial_climático_viñedo/rejilla.json


701 features, 44875 bytes of geometry, 32 bytes of separate metadata, 4012 bytes of string pool


Task created
Finished processing


  99.9%  10/497/381  


##  Duración de las sequías y riesgo de incendios en zonas de dehesa.

In [ ]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Indicadores_dehesa'
names = ['fire_danger', 'dry_spells']

for name in names:
    # Create MBTiles
    print('Creating MBTiles')
    source_path = f"{path}/{folder_name}/dehesa_{name}.json"
    mbtil_path = f"{path}/{folder_name}/dehesa_{name}.mbtiles"
    create_mbtiles(source_path, mbtil_path, folder_name, opts="-Z1 -z10 --force")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    print('Converting from MBTiles to pbf')
    output_path = f"{path}/{folder_name}/dehesa_{name}"
    mbtile_to_pbf(mbtil_path, output_path)

    # Change folder permission
    os.chmod(f"{path}/{folder_name}/dehesa_{name}", 0o777)

    # Rename files from `.pbf` to `.vector.pbf.gz`
    for root, dirs, files in os.walk(f"{path}/{folder_name}/dehesa_{name}"):
        for file in files:
            if file.endswith(".pbf"):
                in_file = os.path.join(root,file)
                os.rename(in_file, in_file.replace('.pbf', '.vector.pbf.gz'))

    # gunzip folder
    cmd = f"gunzip -r {path}/{folder_name}/dehesa_{name}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    load_dotenv()

    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/dehesa_{name}"
    destination_blob_path = f"MBTiles/{folder_name}/"

    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**Upload files to GCS**

In [ ]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Indicadores_dehesa'
names = ['fire_danger', 'dry_spells']

for name in names:
    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    load_dotenv()

    bucket_name = 'ecf-agricultural-climate-impact'
    local_path = f"{path}/{folder_name}/dehesa_{name}"
    destination_blob_path = f"MBTiles/{folder_name}/"

    upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [ ]:
path = '../../datasets/processed/MBTiles'
folder_name = 'Indicadores_dehesa'
name = 'Mapa_cultivos'

vector_tile_processing(path, name, folder_name, gdf, pbf_format=True, upload_to_GCS=True)

## Proyecciones de rendimiento de cereales.
**Create all tiles**

In [ ]:
dataset = 'cereal_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento_cereal'
geo_type = 'Comunidades autonomas'

# Read data
gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type, filter_type = 'cereal')

# Reorganice data
gdf = reorganize_data_new(gdf, geo_type, filter_type = 'cereal') 

# Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format
vector_tile_processing(path, dataset, folder_name, gdf, geo_type, pbf_format=True, upload_to_GCS=True)